In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


LẬT ẢNH

In [ ]:
import cv2
from ultralytics import YOLO


def flip_image(image_path, output_path, output_label, x):
    # Đọc ảnh
    image = cv2.imread(image_path)

    if image is None:
        print("Không thể đọc ảnh.")
        return

    # Lật ngược ảnh từ trái sang phải
    flipped_image = cv2.flip(image, 1)

    # Lưu ảnh đã lật ngược
    cv2.imwrite(output_path, flipped_image)

    results = model(source=output_path, conf=0.60, save=True)

    for i in range(len(results[0].boxes)):

                xb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][0]
                yb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][1]
                wb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][2]

                hb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][3]

                a = results[0].keypoints[i].xyn.clone().detach().cpu().numpy()[0]

                print("----------------------------------------------")

                # with open(output_label, mode='a') as f:
                #     f.write(f"{id} {xb} {yb} {wb} {hb}")
                #     for i in a:
                #         for j in i:
                #             f.write(f" {j}")
                #     f.write("\n")

                with open(output_label, mode='a') as f:
                    f.write(f"{id} {xb} {yb} {wb} {hb}")
                    for i in range(0,13):
                        for j in a[i]:
                            f.write(f" {j}")
                    f.write("\n")

                f.close()


    print("Đã lật ngược ảnh thành công.")
    makeClass(output_path, f"{output_path[:-4]}x.jpg",f"{output_label[:-4]}x.txt", x)

# if __name__ == "__main__":
#     input_image_path = "/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/Draft/img/1.jpg"  # Thay đổi đường dẫn tới ảnh của bạn
#     output_image_path = "/content/flipped_image.jpg"  # Thay đổi đường dẫn đến nơi bạn muốn lưu ảnh đã lật ngược

#     flip_image(input_image_path, output_image_path)


NHÂN BẢN

In [ ]:
import cv2
import numpy as np
import random
from ultralytics import YOLO


def clone_image(image, x):
    # Sao chép ảnh ban đầu
    cloned_images = [image.copy() for _ in range(x*x)]
    return cloned_images

def modify_images(images):
    # Chọn ngẫu nhiên một số ảnh để sửa đổi
    num_images_to_modify = random.randint(1, len(images) - 1)
    modified_indices = random.sample(range(len(images)), num_images_to_modify)

    for index in modified_indices:
        # Chọn ngẫu nhiên một phép biến đổi
        modification = random.choice(["flip", "resize"])

        if modification == "flip":
            # Lật ngang ảnh từ trái sang phải
            images[index] = cv2.flip(images[index], 1)
        elif modification == "resize":
            # Thu nhỏ ảnh đi một tỉ lệ ngẫu nhiên
            scale_factor = random.uniform(0.5, 0.9)
            new_width = int(images[index].shape[1] * scale_factor)
            new_height = int(images[index].shape[0] * scale_factor)
            images[index] = cv2.resize(images[index], (new_width, new_height))

    return images

def makeClass(input_image_path, output_image_path,output_label, x):
    # Đọc ảnh đầu vào
    original_image = cv2.imread(input_image_path)

    if original_image is None:
        print("Không thể đọc ảnh.")
        return

    # Nhân bản ảnh
    cloned_images = clone_image(original_image,x)

    # Sửa đổi một số ảnh
    modified_images = modify_images(cloned_images)

    # Lấy kích thước của ảnh đầu tiên
    height, width, _ = modified_images[0].shape

    # Chia thành 3 hàng và 3 cột
    rows = x
    cols = x
    # Tạo một list để chứa 3 hàng ảnh
    row_images = []
    for i in range(0, len(modified_images), cols):
        # Resize tất cả các ảnh về kích thước của ảnh đầu tiên
        resized_images = [cv2.resize(img, (width, height)) for img in modified_images[i:i+cols]]
        row_images.append(np.hstack(resized_images))

    # Ghép lại các hàng ảnh để tạo thành một ảnh duy nhất
    merged_image = np.vstack(row_images)

    # Lưu ảnh ghép
    cv2.imwrite(output_image_path, merged_image)

    # Hiển thị ảnh ghép
    # cv2.imshow("Merged Image", merged_image)
    resize_image(output_image_path, output_image_path)


    results = model(source=output_image_path,conf=0.5, save=True)

    for i in range(len(results[0].boxes)):

                xb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][0]
                yb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][1]
                wb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][2]

                hb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][3]

                a = results[0].keypoints[i].xyn.clone().detach().cpu().numpy()[0]

                print("----------------------------------------------")

                # with open(output_label, mode='a') as f:
                #     f.write(f"{id} {xb} {yb} {wb} {hb}")
                #     for i in a:
                #         for j in i:
                #             f.write(f" {j}")
                #     f.write("\n")

                with open(output_label, mode='a') as f:
                    f.write(f"{id} {xb} {yb} {wb} {hb}")
                    for i in range(0,13):
                        for j in a[i]:
                            f.write(f" {j}")
                    f.write("\n")

                f.close()

# if __name__ == "__main__":
#     input_image_path = 'D:\\Project\\NCKH_Gian_lan\\code\\Draft_06_12\\detected_frames\\1610.jpg'
#     output_image_path = 'D:\\Project\\NCKH_Gian_lan\\code\\Draft_06_12\\detected_frames\\1610_merged.jpg'

#     main(input_image_path, output_image_path)


MAIN


In [ ]:
import cv2
from PIL import Image
import os
from ultralytics import YOLO
import torch

def cut_and_resize_video(video_path, output_folder, max_size, frames_per_second, dem):
    print('max_size: ', max_size)
    print('frames_per_second: ', frames_per_second)
    # dem = 0  # Bắt đầu với số 33 (đúng theo mã số đặt trong câu hỏi của bạn)

    # Tạo thư mục đầu ra nếu nó không tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Mở video
    cap = cv2.VideoCapture(video_path)

    # Lấy tần suất khung hình của video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Tính toán bước nhảy giữa các khung hình
    frame_skip = fps // frames_per_second

    # Đọc và cắt từng khung hình
    frame_count = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Lưu khung hình mỗi frame_skip
        if frame_count % frame_skip == 0:

            # Lưu ảnh vào thư mục tạm thời
            temp_image_path = os.path.join(output_folder, f"temp_frame.jpg")
            cv2.imwrite(temp_image_path, frame)

            # Mở ảnh và giảm kích thước không thay đổi tỷ lệ
            img = Image.open(temp_image_path)
            img.thumbnail((max_size, max_size))

            # Tạo một ảnh mới với kích thước chính xác là 640x640
            new_img = Image.new("RGB", (max_size, max_size), (255, 255, 255))
            new_img.paste(img, ((max_size - img.width) // 2, (max_size - img.height) // 2))

            new_img = new_img.convert('L')


            # Lưu ảnh đã giảm kích thước
            dem += 1
            print('STT: ',dem)
            resized_image_path = os.path.join(output_folder, f"{dem}.jpg")
            new_img.save(resized_image_path)

            # Xóa ảnh tạm thời
            os.remove(temp_image_path)


            x=0
            if dem%2==0:
              x=2
            else:
              x=3
            #Nhân bản
            makeClass(f"{output_folder}/{dem}.jpg", f"{output_folder}/{str(dem)+'x'}.jpg",f"{labels}/{dem}x.txt", x)

            #Lật ảnh
            flip_image(f"{output_folder}/{dem}.jpg", f"{output_folder}/{str(dem)+'L'}.jpg", f"{labels}/{dem}L.txt", x)





            results = model(source=f"{output_folder}/{dem}.jpg", conf=0.60, save=True)

            for i in range(len(results[0].boxes)):

                xb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][0]
                yb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][1]
                wb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][2]

                hb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][3]

                a = results[0].keypoints[i].xyn.clone().detach().cpu().numpy()[0]

                print("----------------------------------------------")

                with open(f"{labels}/{dem}.txt", mode='a') as f:
                    f.write(f"{id} {xb} {yb} {wb} {hb}")
                    for i in range(0,13):
                        for j in a[i]:
                            f.write(f" {j}")
                    f.write("\n")


                # with open(f"{labels}/{dem}.txt", mode='a') as f:
                #     f.write(f"-> {id} {xb} {yb} {wb} {hb}")
                #     for i in a:
                #         for j in i:
                #             f.write(f" {j}")
                #     f.write("\n")

                f.close()










        frame_count += 1


    cap.release()

model = YOLO("yolov8x-pose.pt")

video_path = input("Nhập địa chỉ video: ")
output_folder = input("nhập folder chứa ảnh: ")
max_size = 640

num_of_imgs=int(input("Nhập số lượng ảnh: "))
frames_per_second = num_of_imgs//get_video_length(video_path)//4

# frames_per_second = int(input('Nhập số ảnh trong 1 giây: '))
dem = int(input("nhập số bắt đầu đêm: "))
# end = int(input("Nhập số kết thúc: "))

labels=input('Duong dan folder label: ')

id = int(input('id: '))


# video_path = f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/video/quay ngang 22-03 - Được tạo bằng Clipchamp_1711113799935.mp4'
# output_folder = f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/Draft/img1'
# max_size = 640
# frames_per_second = 1
# dem = 0
# end = 100

# labels=f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/Draft/label1'

# id = 0


cut_and_resize_video(video_path, output_folder, max_size, frames_per_second, dem)

print("----------------------------------------------------------------------------------------------------------")








In [ ]:

import cv2
from PIL import Image
import os
from ultralytics import YOLO
import torch

def cut_and_resize_video(video_path, output_folder, max_size, frames_per_second, dem):
    print('max_size: ', max_size)
    print('frames_per_second: ', frames_per_second)
    # dem = 0  # Bắt đầu với số 33 (đúng theo mã số đặt trong câu hỏi của bạn)

    # Tạo thư mục đầu ra nếu nó không tồn tại
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Mở video
    cap = cv2.VideoCapture(video_path)

    # Lấy tần suất khung hình của video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Tính toán bước nhảy giữa các khung hình
    frame_skip = fps // frames_per_second

    # Đọc và cắt từng khung hình
    frame_count = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Lưu khung hình mỗi frame_skip
        if frame_count % frame_skip == 0:

            # Lưu ảnh vào thư mục tạm thời
            temp_image_path = os.path.join(output_folder, f"temp_frame.jpg")
            cv2.imwrite(temp_image_path, frame)

            # Mở ảnh và giảm kích thước không thay đổi tỷ lệ
            img = Image.open(temp_image_path)
            img.thumbnail((max_size, max_size))

            # Tạo một ảnh mới với kích thước chính xác là 640x640
            new_img = Image.new("RGB", (max_size, max_size), (255, 255, 255))
            new_img.paste(img, ((max_size - img.width) // 2, (max_size - img.height) // 2))

            new_img = new_img.convert('L')


            # Lưu ảnh đã giảm kích thước
            dem += 1
            print('STT: ',dem)
            resized_image_path = os.path.join(output_folder, f"{dem}.jpg")
            new_img.save(resized_image_path)

            # Xóa ảnh tạm thời
            os.remove(temp_image_path)


            x=0
            if dem%2==0:
              x=2
            else:
              x=2
            #Nhân bản
            makeClass(f"{output_folder}/{dem}.jpg", f"{output_folder}/{str(dem)+'x'}.jpg",f"{labels}/{dem}x.txt", x)

            #Lật ảnh
            flip_image(f"{output_folder}/{dem}.jpg", f"{output_folder}/{str(dem)+'L'}.jpg", f"{labels}/{dem}L.txt", x)





            results = model(source=f"{output_folder}/{dem}.jpg", conf=0.60, save=True)

            for i in range(len(results[0].boxes)):

                xb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][0]
                yb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][1]
                wb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][2]

                hb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][3]

                a = results[0].keypoints[i].xyn.clone().detach().cpu().numpy()[0]

                print("----------------------------------------------")

                with open(f"{labels}/{dem}.txt", mode='a') as f:
                    f.write(f"{id} {xb} {yb} {wb} {hb}")
                    for i in range(0,13):
                        for j in a[i]:
                            f.write(f" {j}")
                    f.write("\n")


                # with open(f"{labels}/{dem}.txt", mode='a') as f:
                #     f.write(f"-> {id} {xb} {yb} {wb} {hb}")
                #     for i in a:
                #         for j in i:
                #             f.write(f" {j}")
                #     f.write("\n")

                f.close()










        frame_count += 1


    cap.release()

model = YOLO("yolov8x-pose.pt")

video_path = input("Nhập địa chỉ video: ")
output_folder = input("nhập folder chứa ảnh: ")
max_size = 640

num_of_imgs=int(input("Nhập số lượng ảnh: "))
frames_per_second = num_of_imgs//get_video_length(video_path)//4

# frames_per_second = int(input('Nhập số ảnh trong 1 giây: '))
dem = int(input("nhập số bắt đầu đêm: "))
# end = int(input("Nhập số kết thúc: "))

labels=input('Duong dan folder label: ')

id = int(input('id: '))


# video_path = f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/video/quay ngang 22-03 - Được tạo bằng Clipchamp_1711113799935.mp4'
# output_folder = f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/Draft/img1'
# max_size = 640
# frames_per_second = 1
# dem = 0
# end = 100

# labels=f'/content/drive/MyDrive/Colab Notebooks/Gian_lan_thi_cu/Draft/label1'

# id = 0


cut_and_resize_video(video_path, output_folder, max_size, frames_per_second, dem)

print("----------------------------------------------------------------------------------------------------------")








Nhập địa chỉ video: /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/video/video 19 04/abc quay ngang‐ Được tạo bằng Clipchamp.mp4
nhập folder chứa ảnh: /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train
Nhập số lượng ảnh: 200
nhập số bắt đầu đêm: 2918
Duong dan folder label: /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels/train
id: 2
max_size:  640
frames_per_second:  1.0
STT:  2919

image 1/1 /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train/2919x.jpg: 640x640 4 persons, 22.3ms
Speed: 1.5ms preprocess, 22.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/pose/predict2
----------------------------------------------
----------------------------------------------
----------------------------------------------
----------------------------------------------

image 1/1 /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train/2919L.jpg: 640x640 1 person, 22.5ms
Speed: 1.5ms preprocess, 22.5ms inference, 

In [ ]:
4200
nghiem tuc train: 1->1012
  4048
  4029
quay cop train: 1013-> 1970
  7880
  7845
quay ngang train: 1971-> 2885
  11540
  11469
quay cop abc train: 2886-> 2918
  11672
  11601
quay ngang abc train: 2919-> 2954
  11816
  11723




1100
nghiem tuc val: 1->266
  1064
  1053
quay cop val: 267->540
  2160
  2146
quay ngang val: 541->748
  2992
  2971

4200

In [ ]:
id0: 1114 Nghiem tuc
id1: 2186 quay cop

4300
- 1-> 1114: id0 (nghiem tuc) i4456,l4434
- 1115 -> 2186: id1 (quay cop) i4288
- 2187 -> 3119: id2
- 3120 ->3160: id1
- 3161-> :id2

- 1->933: id2 (3732) quay ngang
- 934->1827: id1 (3576) quay cop
- 1828->2941: id0 ()
11764
11295

- 1->229:0 phúc nhân
- 230->379:0  Tuấn +=1516, 1489 nghiem tuc
- 380->705:1 += 2820,2777 quay cop
- 706->1086: 2+= 4344,4292

- 1: quay cop: 621
- 2: quay ngang: 546
- 0: nt: 545

RESIZE TO 640PX

In [ ]:
from PIL import Image
import cv2
import os

def resize_image(input_file_path, output_file_path):
    # Tạo một ảnh tạm thời
    temp_image_path = os.path.join("/content", "temp_frame.jpg")
    frame = cv2.imread(input_file_path)
    cv2.imwrite(temp_image_path, frame)

    # Thiết lập kích thước tối đa cho ảnh mới
    max_size = 640

    # Mở ảnh và giảm kích thước không thay đổi tỷ lệ
    img = Image.open(temp_image_path)
    img.thumbnail((max_size, max_size))

    # Tạo một ảnh mới với kích thước chính xác là 640x640
    new_img = Image.new("RGB", (max_size, max_size), (255, 255, 255))
    new_img.paste(img, ((max_size - img.width) // 2, (max_size - img.height) // 2))

    # Lưu ảnh đã chuyển đổi vào đường dẫn output_file_path
    new_img.save(output_file_path)

    # Xóa ảnh tạm thời
    os.remove(temp_image_path)

# Sử dụng hàm
input_file = "/content/drive/MyDrive/Colab Notebooks/13x.jpg"
output_file = "/content/drive/MyDrive/Colab Notebooks/13x.jpg"
resize_image(input_file, output_file)


LẤY ĐỘ DÀI VIDEO

In [ ]:
import cv2

def get_video_length(video_path):
    # Đọc video
    video_capture = cv2.VideoCapture(video_path)

    # Lấy tổng số khung hình
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Lấy FPS (số khung hình trên mỗi giây)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Tính độ dài của video
    video_length_seconds = total_frames / fps

    # Giải phóng tài nguyên
    video_capture.release()

    return (video_length_seconds//1)

# Sử dụng hàm
video_path = "/content/drive/MyDrive/Colab Notebooks/NCKH_Test/Video/Vuong_quay_ngang - Được tạo bằng Clipchamp_1712222940850.mp4"
video_length_seconds = get_video_length(video_path)
print("Độ dài của video là:", video_length_seconds//1, "giây")


XÓA FILE

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Thư mục chứa ảnh trên Google Drive
folder_path = input("fuck: ")
# Lặp qua các tệp trong thư mục
for file_name in os.listdir(folder_path):

        # Lấy số thứ tự từ tên tệp
        # if file_name[-4:]=="":
          file_path = os.path.join(folder_path, file_name)
              # Xóa tệp
          os.remove(file_path)
          print(f"Đã xóa: {file_name[-4:]}")

In [ ]:
import re

def xoa_va_chuyen_sang_so_nguyen(chuoi):
    # Loại bỏ tất cả các kí tự không phải số
    chuoi_so = re.sub(r'\D', '', chuoi)
    # Chuyển chuỗi thành kiểu số nguyên
    so_nguyen = int(chuoi_so)
    return so_nguyen

# Ví dụ
chuoi = "abc123def456ghi"
so_nguyen = xoa_va_chuyen_sang_so_nguyen(chuoi)
print(so_nguyen)


123456


**So sanh img va label**

In [ ]:
import os

# # Mount Google Drive
# drive.mount('/content/drive')

# Thư mục chứa ảnh trên Google Drive
folder_path1 = "/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train"

folde2 = "/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels/train"
dem =0
# Lặp qua các tệp trong thư mục
# for file_name in os.listdir(folder_path1):
#   if f"{file_name[:-4]}.txt" not in os.listdir(folde2):
#     dem+=1
#     print(file_name)
# print("Tong: ", dem)

for i in range(3119,0,-1):
  print(i)
  if f"{i}.jpg" in os.listdir(folder_path1) and f"{i}.txt" not in os.listdir(folde2):
    dem+=1
    print(f"{i}.jpg")
  if f"{i}x.jpg" in os.listdir(folder_path1) and f"{i}x.txt" not in os.listdir(folde2):
    dem+=1
    print(f"{i}x.jpg")
  if f"{i}L.jpg" in os.listdir(folder_path1) and f"{i}L.txt" not in os.listdir(folde2):
    dem+=1
    print(f"{i}L.jpg")
  if f"{i}Lx.jpg" in os.listdir(folder_path1) and f"{i}Lx.txt" not in os.listdir(folde2):
    dem+=1
    print(f"{i}Lx.jpg")

In [ ]:
import os
model = YOLO('yolov8x-pose.pt')

# # Mount Google Drive
# drive.mount('/content/drive')

# Thư mục chứa ảnh trên Google Drive
folder_path1 = "/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train"

folde2 = "/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels/train"
# Lặp qua các tệp trong thư mục
for file_name in os.listdir(folder_path1):
  if f"{file_name[:-4]}.txt" not in os.listdir(folde2):
    print("----------------------->",file_name)
    results = model(source=f"{folder_path1}/{file_name}", conf=0.50, save=True)

    for i in range(len(results[0].boxes)):

                xb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][0]
                yb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][1]
                wb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][2]

                hb = results[0].boxes[i].xywhn.clone().detach().cpu().numpy()[0][3]

                a = results[0].keypoints[i].xyn.clone().detach().cpu().numpy()[0]

                print("----------------------------------------------")

                with open(f"/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/Draft/{file_name[:-4]}.txt", mode='a') as f:
                    f.write(f"{id} {xb} {yb} {wb} {hb}")
                    for i in range(0,13):
                        for j in a[i]:
                            f.write(f" {j}")
                    f.write("\n")

ĐẾM FILE

In [ ]:


import os

def count_files_in_folder(folder_path):
    # Kiểm tra xem folder_path có tồn tại không
    if not os.path.exists(folder_path):
        print("Thư mục không tồn tại.")
        return

    # Khởi tạo biến đếm
    file_count = 0

    # Lặp qua tất cả các tệp trong thư mục
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

print(count_files_in_folder('/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train'))
print(count_files_in_folder('/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels/train'))

11816
11723


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels

In [ ]:
from ultralytics import YOLO

In [ ]:
model=YOLO('yolov8x-pose.pt')
results = model(source='/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train/1841L.jpg',conf=0.6, save=True)


image 1/1 /content/drive/MyDrive/Colab Notebooks/NCKH_13_04/images/train/1841L.jpg: 640x640 (no detections), 22.4ms
Speed: 8.4ms preprocess, 22.4ms inference, 2343.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/pose/predict


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/labels/val-label.zip'

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NCKH_13_04

/content/drive/MyDrive/Colab Notebooks/NCKH_13_04


In [ ]:
!yolo pose train data="/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/mydataset.yaml" model=yolov8n-pose.pt epochs=100 imgsz=640

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/mydataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

In [ ]:
# Resume an interrupted training
!yolo train resume model='/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/runs/pose/train2/weights/last.pt'

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=pose, mode=train, model=/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/runs/pose/train2/weights/last.pt, data=/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/mydataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/runs/pose/train2/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, au

In [ ]:
!yolo pose train data="/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/mydataset.yaml" model=yolov8n-pose.pt epochs=100 imgsz=640


Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=/content/drive/MyDrive/Colab Notebooks/NCKH_13_04/mydataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa